# 数据集生成

## 环境配置读取

In [1]:
import tomllib


def read_toml_config(file_path):
    with open(file_path, "rb") as f:
        config = tomllib.load(f)
    return config

In [2]:
# 读取环境配置
config = read_toml_config("config.toml")

## 配置大模型API

In [3]:
from openai import OpenAI

client = OpenAI(api_key=config.get("API_KEY"), base_url=config.get("BASE_URL"))

包装发送消息的函数

In [4]:
def send_message_to_model(user_content, prompt="You are a helpful assistant", json_type=True):
    format_type = "text"
    if json_type:
        format_type = "json_object"
    response = client.chat.completions.create(
        model=config.get("MODEL_NAME"),
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": user_content},
        ],
        stream=False,
        response_format={
            "type": format_type,
        }
    )
    return response.choices[0].message.content

测试接口状态

In [5]:
test_response = send_message_to_model("Hello", json_type=False)
print(test_response)

Hello! How can I assist you today? 😊


## 行业项目生成

提示词定义

In [6]:
industry_list = [
    "ERP（企业资源计划）",
    "CRM（客户关系管理）",
    "OA（办公自动化）",
    "HRM（人力资源管理）",
    "社交应用",
    "购物",
    "短视频",
    "长视频",
    "嵌入式与物联网",
    "人工智能与数据科学",
    "Web与云服务",
    "科学与工程软件",
    "区块链与金融科技",
    "医疗与生命科学",
    "开发者工具",
]

In [7]:
project_prompt_content = """
请根据用户提供的大致方向，生成对应行业或领域的几个项目类型。

需要生成的一个项目中包含两个部分：

- name：项目的简单名称。
- introduce：简短的减少说明其属于那一个领域和解决的问题。

注意，需要遵守以下规则：

- 生成的项目数需要在{SIZE}左右，请不要生成过多内容。
- 项目的简单介绍请尽可能简洁，请将内容保持在30个字左右。

返回的内容需要为JSON字符串类型，大体如以下格式：
[{{
    "name": "接口自动化系统",
    "introduce": "可以统一管理测试用例，并可以对测试用例进行执行，并提供多种协议的灵活使用。"
}}]
""".format(SIZE=10)

project_prompt_content

'\n请根据用户提供的大致方向，生成对应行业或领域的几个项目类型。\n\n需要生成的一个项目中包含两个部分：\n\n- name：项目的简单名称。\n- introduce：简短的减少说明其属于那一个领域和解决的问题。\n\n注意，需要遵守以下规则：\n\n- 生成的项目数需要在10左右，请不要生成过多内容。\n- 项目的简单介绍请尽可能简洁，请将内容保持在30个字左右。\n\n返回的内容需要为JSON字符串类型，大体如以下格式：\n[{\n    "name": "接口自动化系统",\n    "introduce": "可以统一管理测试用例，并可以对测试用例进行执行，并提供多种协议的灵活使用。"\n}]\n'

In [8]:
import concurrent.futures
import threading
import json

def process_industry(item, index, project_list_lock):
    """处理单个行业的函数"""
    try:
        project_response = send_message_to_model(item, project_prompt_content, json_type=True)
        project_parsed = json.loads(project_response)
        print(f"批次 {index + 1} 生成数量为 {len(project_parsed)}")

        # 使用锁保护共享资源
        with project_list_lock:
            project_list.extend(project_parsed)

        return True
    except Exception as e:
        print(f"处理 {item} 时发生错误: {e}")
        return False

使用多线程处理行业列表，生成项目列表

In [9]:
# 配置线程数
max_workers = 5  # 可以根据需要调整线程数

project_list = []
project_list_lock = threading.Lock()  # 创建锁对象保护共享列表

# 使用线程池执行请求
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    # 创建任务列表
    futures = [
        executor.submit(process_industry, item, index, project_list_lock)
        for index, item in enumerate(industry_list)
    ]

    # 等待所有任务完成
    completed = 0
    for future in concurrent.futures.as_completed(futures):
        if future.result():
            completed += 1

    print(f"完成 {completed}/{len(industry_list)} 个行业的处理")

print(f"总共收集了 {len(project_list)} 个项目")

批次 1 生成数量为 10
批次 3 生成数量为 10
批次 4 生成数量为 10
批次 2 生成数量为 10
批次 5 生成数量为 10
批次 6 生成数量为 10
批次 9 生成数量为 10
批次 8 生成数量为 10
批次 7 生成数量为 10
批次 10 生成数量为 10
批次 11 生成数量为 10
批次 12 生成数量为 10
批次 13 生成数量为 10
批次 15 生成数量为 10
批次 14 生成数量为 10
完成 15/15 个行业的处理
总共收集了 150 个项目


保存项目列表到CSV文件

In [10]:
import pandas as pd

df = pd.DataFrame(project_list)
df.to_csv("cache/projects.csv", index=False, columns=["name", "introduce"])

## 数据集生成

### 提示词定义

In [21]:
dataset_prompt_content = """
当前我需要你帮我生成一个数据集，这个数据集将会用于微调大语言模型。数据集的作用是实现一个函数名生成器，能够根据给定的项目类型、当前模块、目标名称类型、格式化类型和目标描述等信息，生成相应的函数名、类名和变量名等信息。
在起名时，需要考虑到项目的行业背景、当前模块的功能、目标名称类型和格式化类型等信息，以便生成符合实际需求的名称。

数据集的格式是一个json字符串，包含两个字段：question和answer。其中question字段是一个字符串，包含了以下信息：

- 项目类型：项目的类型、名称等标记项目所属行业或领域的标签。
- 当前模块：当前正在开发的模块或功能的名称。
- 目标名称类型：需要生成的名称类型，如函数名、类名、变量名等。
- 格式化类型：生成名称的格式化方式，如驼峰命名、下划线命名等。

请注意，需要准守一下几点要求：

- 生成的结果的问答对数量应该在{SIZE}个左右，请不要生成过多的问答对。
- 生成的结果应该是一个json字符串，包含两个字段：question和answer。
- 请根据实际中可能会存在的项目进行生成，尽量不要生成一些不符合实际的项目。
- 用户会说明一些行业信息或项目基础信息，请根据用户的说明进行扩展。

我需要你生成的内容是一个json字符串，如下示例：
[{{
    "question": "项目类型：接口自动化系统；项目介绍：可以统一管理测试用例，并可以对测试用例进行执行，并提供多种协议的灵活使用。；当前模块：集合管理模块；目标名称类型：函数名；格式化类型：驼峰命名（首字母小写）；目标描述：获取案例分页列表；生成数量：5；",
    "answer": ["getCaseList", "getCasePage", "getCaseListPage", "getCaseListByPage", "selectCaseListByPage"]
}}]
""".format(SIZE="20")

dataset_prompt_content

'\n当前我需要你帮我生成一个数据集，这个数据集将会用于微调大语言模型。数据集的作用是实现一个函数名生成器，能够根据给定的项目类型、当前模块、目标名称类型、格式化类型和目标描述等信息，生成相应的函数名、类名和变量名等信息。\n在起名时，需要考虑到项目的行业背景、当前模块的功能、目标名称类型和格式化类型等信息，以便生成符合实际需求的名称。\n\n数据集的格式是一个json字符串，包含两个字段：question和answer。其中question字段是一个字符串，包含了以下信息：\n\n- 项目类型：项目的类型、名称等标记项目所属行业或领域的标签。\n- 当前模块：当前正在开发的模块或功能的名称。\n- 目标名称类型：需要生成的名称类型，如函数名、类名、变量名等。\n- 格式化类型：生成名称的格式化方式，如驼峰命名、下划线命名等。\n\n请注意，需要准守一下几点要求：\n\n- 生成的结果的问答对数量应该在20个左右，请不要生成过多的问答对。\n- 生成的结果应该是一个json字符串，包含两个字段：question和answer。\n- 请根据实际中可能会存在的项目进行生成，尽量不要生成一些不符合实际的项目。\n- 用户会说明一些行业信息或项目基础信息，请根据用户的说明进行扩展。\n\n我需要你生成的内容是一个json字符串，如下示例：\n[{\n    "question": "项目类型：接口自动化系统；项目介绍：可以统一管理测试用例，并可以对测试用例进行执行，并提供多种协议的灵活使用。；当前模块：集合管理模块；目标名称类型：函数名；格式化类型：驼峰命名（首字母小写）；目标描述：获取案例分页列表；生成数量：5；",\n    "answer": ["getCaseList", "getCasePage", "getCaseListPage", "getCaseListByPage", "selectCaseListByPage"]\n}]\n'

### 测试数据集生成

In [19]:
import pandas as pd

projects_data = pd.read_csv("cache/projects.csv", encoding="utf-8")

projects_data.shape

(150, 2)

In [14]:
for index, (name, introduce) in projects_data.iterrows():
    print(f"项目名称：{name}，项目介绍：{introduce}")
    break

项目名称：供应链管理，项目介绍：优化企业供应链流程，提高效率并降低成本。
项目类型：自动化平台；当前模块：集合管理模块；目标名称类型：函数名；格式化类型：驼峰命名（首字母小写）；目标描述：获取案例分页列表；生成数量：5；


In [15]:
import concurrent.futures
import threading
import json

def process_dataset(user_content, index, dataset_list_lock):
    """处理单个行业的函数"""
    try:
        dataset_response = send_message_to_model(user_content, dataset_prompt_content, json_type=True)
        dataset_parsed = json.loads(dataset_response)
        print(f"批次 {index + 1} 生成数量为 {len(dataset_parsed)}")

        # 使用锁保护共享资源
        with dataset_list_lock:
            dataset_list.extend(dataset_parsed)

        return True
    except Exception as e:
        print(f"处理 {user_content} 时发生错误: {e}")
        return False

In [22]:
# 配置线程数
max_workers = 10  # 可以根据需要调整线程数

dataset_list = []
dataset_list_lock = threading.Lock()  # 创建锁对象保护共享列表

# 使用线程池执行请求
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    # 创建任务列表
    futures = [
        executor.submit(process_dataset, f"项目名称：{name}，项目介绍：{introduce}", i, dataset_list_lock)
        for i, (name, introduce) in projects_data.iterrows()
    ]

    # 等待所有任务完成
    completed = 0
    for future in concurrent.futures.as_completed(futures):
        if future.result():
            completed += 1

    print(f"完成 {completed} 个数据集任务的处理")

print(f"总共收集了 {len(dataset_list)} 个项目")

批次 8 生成数量为 5
批次 9 生成数量为 5
批次 1 生成数量为 10
批次 4 生成数量为 5
批次 2 生成数量为 10
批次 5 生成数量为 10
批次 12 生成数量为 5
批次 14 生成数量为 5
批次 7 生成数量为 10
批次 15 生成数量为 5
批次 13 生成数量为 10
批次 11 生成数量为 5
批次 16 生成数量为 9
批次 17 生成数量为 5
批次 10 生成数量为 20
批次 3 生成数量为 10
批次 20 生成数量为 10
批次 18 生成数量为 5
批次 25 生成数量为 5
批次 26 生成数量为 5
批次 19 生成数量为 10
批次 24 生成数量为 5
批次 28 生成数量为 5
批次 22 生成数量为 5
批次 32 生成数量为 5
批次 31 生成数量为 5
批次 29 生成数量为 10
批次 27 生成数量为 5
批次 21 生成数量为 5
批次 6 生成数量为 21
批次 34 生成数量为 5
批次 35 生成数量为 5
批次 23 生成数量为 10
批次 37 生成数量为 5
批次 38 生成数量为 5
批次 39 生成数量为 5
批次 42 生成数量为 5
批次 40 生成数量为 5
批次 43 生成数量为 5
批次 44 生成数量为 5
批次 33 生成数量为 10
批次 30 生成数量为 15
批次 50 生成数量为 5
批次 48 生成数量为 5
批次 45 生成数量为 10
批次 47 生成数量为 10
批次 53 生成数量为 5
批次 55 生成数量为 5
批次 52 生成数量为 5
批次 49 生成数量为 10
批次 46 生成数量为 5
批次 54 生成数量为 5
批次 41 生成数量为 10
批次 51 生成数量为 10
批次 57 生成数量为 5
批次 59 生成数量为 5
批次 36 生成数量为 20
批次 58 生成数量为 5
批次 56 生成数量为 5
批次 66 生成数量为 5
批次 63 生成数量为 5
批次 60 生成数量为 10
批次 67 生成数量为 5
批次 65 生成数量为 5
批次 61 生成数量为 10
批次 71 生成数量为 5
批次 69 生成数量为 10
批次 75 生成数量为 5
批次 73 生成数量为 10
批次 70 生成数量为 10
批次 6

In [23]:
import pandas as pd

table_dataset_list = []
for item in dataset_list:
    table_dataset_list.append({
        "question": item["question"],
        "answer": "|".join(item["answer"])
    })
df = pd.DataFrame(table_dataset_list)
df.to_csv("cache/nomist_dataset.csv", index=False, columns=["question", "answer"])

In [ ]:
from huggingface_hub import HfApi

api = HfApi(token=config.get("HF_TOKEN"))
api.upload_folder(
    folder_path="./cache/",
    repo_id="MoChenYa/code-nomist-llm-dataset",
    repo_type="dataset",
)